# Detect and trim images

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/ears/annotations/recognition/ids.csv', names=['filename', 'label'])
df['label'] = df['label'].astype(str)
train_labels=df[df['filename'].str.startswith('train')]['label'].to_numpy()
test_labels=df[df['filename'].str.startswith('test')]['label'].to_numpy()
df_train = df[df['filename'].str.startswith('train')]
df_test = df[df['filename'].str.startswith('test')]

In [29]:
import cv2
import numpy as np
import glob
import os
from pathlib import Path
import json

class TrimImages:


    def get_annotations(self, annot_name):
            with open(annot_name) as f:
                lines = f.readlines()
                annot = []
                for line in lines:
                    l_arr = line.split(" ")[1:5]
                    l_arr = [float(i) for i in l_arr]
                    annot.append(l_arr)
            return annot
    def run_trim(self):

        from yolov5 import detect
        results = []
        for src_type in ['train', 'test']:
            count = 0
            trim_src = f'data/ears_yolo/images/{src_type}_hist_gauss'
            trim_target = f'data_cnn_test/ears/{src_type}'
            im_list = sorted(glob.glob(trim_src + '/*.png', recursive=True))
            for im_name in im_list:
                # Read an image
                img = cv2.imread(im_name)
                filename = os.path.basename(im_name)
                print('>>>>>>>>>>>>>> Processing: ', src_type, filename)

                # Apply some preprocessing
                # img = preprocess.histogram_equlization_rgb(img) # This one makes VJ worse
                
                # Run the detector. It runs a list of all the detected bounding-boxes. In segmentor you only get a mask matrices, but use the iou_compute in the same way.
                # prediction_list = cascade_detector.detect(img)
                
                prediction_list = detect.run(weights='results/hist_gauss_4_best.pt', source=im_name, nosave=True, save_txt=True)

                # Read annotations:
                prediction_last = glob.glob('runs/detect/*', recursive=True)
                prediction_last.sort(key=os.path.getmtime)
                prediction_name = os.path.join(prediction_last.pop(), 'labels', Path(os.path.basename(im_name)).stem) + '.txt'
                prediction_list = []
                if (os.path.exists(prediction_name)):
                    prediction_list = self.get_annotations(prediction_name)

                # print(prediction_list, annot_list)
                x = len(img[0])
                y = len(img)

                prediction_list_2 = []
                for item in prediction_list:
                    prediction_list_2.append([int(item[0] * x), int(item[1] * y), int(item[2] * x), int(item[3] * y)])
                
                prediction_list_2 = prediction_list_2[0]
                center_x = prediction_list_2[0]
                center_y = prediction_list_2[1]
                width = prediction_list_2[2]
                height = prediction_list_2[3]
                # if (width == 0 or height == 0):
                #     continue
                x_start = center_x - (width // 2)
                y_start = center_y - (height // 2)
                x_end = center_x + (width // 2)
                y_end = center_y + (height // 2)
                trimmed_img = img[y_start:y_end, x_start:x_end]
                if (len(trimmed_img) == 0):
                    continue
                # cv2.imshow('trim', trimmed_img)
                # cv2.waitKey(0)
                # break
                print("Writing result to: "+ os.path.join(trim_target, filename))
                cv2.imwrite(os.path.join(trim_target, filename), trimmed_img)
                count = count + 1
            text = f"Detection success for '{src_type}': {count}/{len(im_list)}"
            results.append(text)
            print(text)
        print(results)
            
            

ev = TrimImages()
ev.run_trim()

YOLOv5  2021-12-12 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 270 layers, 7022326 parameters, 0 gradients, 15.8 GFLOPs


>>>>>>>>>>>>>> Processing:  train 0001.png
image 1/1 D:\FRI\Magisterij\SB\Assignment 3\Faces - Keras\data\ears_yolo\images\train_hist_gauss\0001.png: 

YOLOv5  2021-12-12 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 270 layers, 7022326 parameters, 0 gradients, 15.8 GFLOPs


480x640 1 ear, Done. (0.132s)
Speed: 0.0ms pre-process, 132.0ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp1622
1 labels saved to runs\detect\exp1622\labels
Writing result to: data_cnn_test/ears/train\0001.png
>>>>>>>>>>>>>> Processing:  train 0002.png


KeyboardInterrupt: 

# Convert to torch file structure

In [10]:
import numpy as np
import pandas as pd
import os
from shutil import copyfile
import torch
from sklearn.model_selection import train_test_split

source_dir = 'data_cnn/ears'
target_dir = 'data_torch'
df = pd.read_csv('data_cnn/ears/annotations/recognition/ids.csv', names=['filename', 'label'])
df['label'] = df['label'].astype(str)
train_labels=df[df['filename'].str.startswith('train')]['label'].to_numpy()
test_labels=df[df['filename'].str.startswith('test')]['label'].to_numpy()
df_train = df[df['filename'].str.startswith('train')]
df_test = df[df['filename'].str.startswith('test')]

train_len = int(0.7*len(df_train))
valid_len = len(df_train) - train_len
train, val = train_test_split(df_train, test_size=0.3, random_state=42, shuffle=True)
print("train", train)
print("val", val)


train            filename label
632  train/0383.png    56
657  train/0408.png    59
485  train/0236.png    39
706  train/0457.png    65
459  train/0210.png    36
..              ...   ...
321  train/0072.png    19
356  train/0107.png    23
520  train/0271.png    43
685  train/0436.png    63
352  train/0103.png    23

[525 rows x 2 columns]
val            filename label
756  train/0507.png    71
607  train/0358.png    53
383  train/0134.png    27
500  train/0251.png    41
549  train/0300.png     5
..              ...   ...
432  train/0183.png    32
819  train/0570.png    79
514  train/0265.png    42
998  train/0749.png     7
473  train/0224.png    37

[225 rows x 2 columns]


In [7]:
for index, row in train.iterrows():
    filename = row['filename'].split("/")[-1]
    print("Processing: "+ row['filename'])
    if (not os.path.exists(os.path.join(target_dir, 'test', row['label']))):
        os.mkdir(os.path.join(target_dir, 'val', row['label']))

    src_file = os.path.join(source_dir, row['filename'])
    trgt_file = os.path.join(target_dir, 'train', row['label'], filename)
    copyfile(src_file, trgt_file)

for index, row in val.iterrows():
    filename = row['filename'].split("/")[-1]
    print("Processing: "+ row['filename'])
    if (not os.path.exists(os.path.join(target_dir, 'test', row['label']))):
        os.mkdir(os.path.join(target_dir, 'val', row['label']))

    src_file = os.path.join(source_dir, row['filename'])
    trgt_file = os.path.join(target_dir, 'val', row['label'], filename)
    copyfile(src_file, trgt_file)
    

for index, row in df_test.iterrows():
    filename = row['filename'].split("/")[-1]
    print("Processing: "+ row['filename'])
    if (not os.path.exists(os.path.join(target_dir, 'test', row['label']))):
        os.mkdir(os.path.join(target_dir, 'test', row['label']))

    src_file = os.path.join(source_dir, row['filename'])
    trgt_file = os.path.join(target_dir, 'test', row['label'], filename)
    copyfile(src_file, trgt_file)
    
    

Processing: 0001.png
Processing: 0002.png
Processing: 0003.png
Processing: 0004.png
Processing: 0005.png
Processing: 0006.png
Processing: 0007.png
Processing: 0008.png
Processing: 0009.png
Processing: 0010.png
Processing: 0011.png
Processing: 0012.png
Processing: 0013.png
Processing: 0014.png
Processing: 0015.png
Processing: 0016.png
Processing: 0017.png
Processing: 0018.png
Processing: 0019.png
Processing: 0020.png
Processing: 0021.png
Processing: 0022.png
Processing: 0023.png
Processing: 0024.png
Processing: 0025.png
Processing: 0026.png
Processing: 0027.png
Processing: 0028.png
Processing: 0029.png
Processing: 0030.png
Processing: 0031.png
Processing: 0032.png
Processing: 0033.png
Processing: 0034.png
Processing: 0035.png
Processing: 0036.png
Processing: 0037.png
Processing: 0038.png
Processing: 0039.png
Processing: 0040.png
Processing: 0041.png
Processing: 0042.png
Processing: 0043.png
Processing: 0044.png
Processing: 0045.png
Processing: 0046.png
Processing: 0047.png
Processing: 0